In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [ ]:
from __future__ import print_function, division

import torch 
import torch.nn as nn
import torchvision.datasets as dsets
from torch.autograd import Variable
import numpy as np
import pandas as pd
import sys
import math
import time
from tqdm import *
import torchvision

from torchvision.models import resnet34, resnet50

from data_preprocess import *
from utils import *
from logger import Logger



import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import torchvision.transforms as transforms

plt.ion()

In [ ]:
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')
batch_size = 20

In [ ]:
class ImageDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.label_table = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.label_table)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir, self.label_table.ix[idx, 0])
        image = io.imread(img_name)
        labels = self.label_table.ix[idx, 1:]
        #label_table = np.array([label_table])
        #landmarks = landmarks.astype('float').reshape(-1, 2)
        sample = {'image': image, 'labels': labels}

        if self.transform:
            sample = self.transform(sample)

        return sample


In [ ]:
class Rescale(object):
    """Rescale the image in a sample to a given size.
    Args:
        output_size (tuple or tuple): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        image, labels = sample['image'], sample['labels']
        h, w = image.shape[:2]
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size
        new_h, new_w = int(new_h), int(new_w)
        img = transform.resize(image, (new_h, new_w))


        return {'image': img, 'labels': labels}


class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, labels = sample['image'], sample['labels']


        image = image.transpose((2, 0, 1))
        image = torch.from_numpy(image).float()
        image = image[0:3]

        in_transform = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        image = in_transform(image)[:3,:,:]

        return image, torch.Tensor(labels)


In [ ]:
train_set = ImageDataset(csv_file='data/ch1_train.csv',
        root_dir='data',
        transform=transforms.Compose([
            Rescale(256),
            ToTensor(),
            ]))

validation_set = ImageDataset(csv_file='data/ch1_val.csv',
        root_dir='data',
        transform=transforms.Compose([
            Rescale(256),
            ToTensor(),
            ]))


train_loader = torch.utils.data.DataLoader(dataset=train_set,
                                           shuffle=True,
                                           batch_size=batch_size)

valid_loader = torch.utils.data.DataLoader(dataset=validation_set,
                                           shuffle=True,
                                          batch_size=batch_size)
classes = ['0','1']


In [ ]:
model = models.resnet34(pretrained=True)
model = model.to('cuda')
for param in model.parameters():
    param.requires_grad = False
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
  nn.Dropout(0.25),
  nn.Linear(num_ftrs, 1024),
  nn.ReLU(),
  nn.Dropout(0.25),
  nn.Linear(1024, 1024),
  nn.ReLU(),
  nn.Dropout(0.25),
  nn.Linear(1024, 2)
)
model = model.to('cuda')
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.04)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
n_epochs = 10
count = 0
valid_loss_min = np.Inf 

for epoch in range(1, n_epochs+1):

    train_loss = 0.0
    valid_loss = 0.0

    model.train()
    for data, target in train_loader:

        data = data.float()
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        
        output = model(data)
        target = target.squeeze(1)
        target = target.type(torch.LongTensor).cuda()
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
        count += 1
        

    model.eval()
    for data, target in valid_loader:
        data = data.float()
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        output = model(data)
        target = target.squeeze(1)
        target = target.type(torch.LongTensor).cuda()
        loss = criterion(output, target)
        valid_loss += loss.item()*data.size(0)
    
    train_loss = train_loss/len(train_loader.sampler)
    valid_loss = valid_loss/len(valid_loader.sampler)
        
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'res-yahoo.pt')
        valid_loss_min = valid_loss

In [ ]:
torch.save(model, 'res_mod.pt')